In [126]:
# Importing necessary libraries
import pandas as pd
import numpy as np

In [127]:
# Load CSV
steam_games = pd.read_csv("../data/games.csv")

## Basic EDA

In [128]:
steam_games.head(5)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


## Correcting Feature names
It seems like the feature names have been named incorrectly

In [102]:
steam_games.columns = ['Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'Discount', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']

In [103]:
steam_games.head(3)

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [104]:
steam_games["Categories"].count()

103886

In [105]:
steam_games["Is_Game"] = steam_games["Categories"].fillna('').apply(
    lambda x: "single-player" in x.lower() or "multi-player" in x.lower()
)
steam_games = steam_games[steam_games["Is_Game"] == True].copy()

In [134]:
temp_games[temp_games["Name"] == "Stream Avatars"]

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game,Supported languages,...,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies,Is_Game
665300,Stream Avatars,"Jul 26, 2017",100000 - 200000,1387,0,14.99,0,1,Stream Avatars is an interactive overlay for l...,['English'],...,5174,1005,Travis Miller,Travis Miller,"Steam Workshop,Includes level editor","Animation & Modeling,Audio Production,Design &...","Video Production,Animation & Modeling,Design &...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,False


### Dropping unneeded columns

In [107]:
steam_games.drop(["Screenshots",  "Movies", "Tags", "Website", "Support email", "Header image", "About the game" , "Reviews" , "Metacritic url", "Support url", "Score rank", "Notes", "Reviews", "About the game"], axis=1, inplace=True)

### Dropping all rows with Nulls

In [108]:
null_cols = steam_games.isnull().sum()
null_cols = null_cols[null_cols > 0]
null_cols

Name             6
Developers    6475
Publishers    6778
Categories    7566
Genres        6440
dtype: int64

In [109]:
steam_games.dropna(inplace=True)

### Date Conversion

In this dataset, the release dates are not in the correct format.

In [110]:
steam_games["Release date"].dtype

dtype('O')

In [111]:
pd.to_datetime(steam_games["Release date"], errors='coerce')

20200     2008-10-21
655370    2017-10-12
1732930   2021-11-17
1355720   2020-07-23
1139950   2020-02-03
             ...    
3600970   2025-04-10
3543710   2025-04-11
3265370   2025-04-01
3423620   2025-03-11
3183790   2025-04-10
Name: Release date, Length: 103219, dtype: datetime64[ns]

## Working with cleaned dataset

In [112]:
steam_games.head(5)

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,Supported languages,Full audio languages,...,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Is_Game
20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,['English'],[],...,0,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports",True
655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,"['English', 'French', 'Italian', 'German', 'Sp...",[],...,0,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie",True
1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,"['English', 'Portuguese - Brazil']",[],...,0,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",True
1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,"['English', 'French', 'Italian', 'German', 'Sp...",[],...,0,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie",True
1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,"['English', 'Spanish - Spain']",[],...,0,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie",True


In [113]:
# Top 10 Peak CCUs
steam_games[['Name', 'Peak CCU']].sort_values(by='Peak CCU', ascending=False).head(10)

,Name,Peak CCU
2246340,Monster Hunter Wilds,1311366
1623730,Palworld,1284268
2358720,Black Myth: Wukong,1214621
990080,Hogwarts Legacy,872138
730,Counter-Strike: Global Offensive,825215
570,Dota 2,558759
1326470,Sons Of The Forest,405191
2694490,Path of Exile 2,373658
1599340,Lost Ark,287501
578080,PUBG: BATTLEGROUNDS,275374


In [114]:
# Most common game prices
steam_games['Price'].value_counts().head(5)

Price
0.00    16323
4.99     9231
9.99     8177
0.99     7340
2.99     6066
Name: count, dtype: int64

In [115]:
steam_games["User score"].dtype

dtype('int64')

In [116]:
steam_games['Estimated owners_str'] = steam_games['Estimated owners'].copy()
owners_split = steam_games["Estimated owners"].astype(str).str.split(' - ', expand=True)
owners_split_numeric = owners_split.apply(pd.to_numeric, errors='coerce')
steam_games["Estimated owners avg"] = owners_split_numeric.mean(axis=1)

In [117]:
steam_games.loc[steam_games["Price"] > 0, "Value"] = steam_games["Average playtime forever"] / steam_games["Price"]


valid_mask = (
    (steam_games["Price"] > 0) &
    np.isfinite(steam_games["Value"]) &
    steam_games["Metacritic score"].notna() &
    (steam_games["Metacritic score"] > 0)
)

steam_games.loc[valid_mask, "Weighted Value"] = steam_games.loc[valid_mask, "Value"] * steam_games.loc[valid_mask, "Metacritic score"]

filtered = steam_games[
    (steam_games["Price"] > 10) &
    (steam_games["Recommendations"] > 10000)
]
filtered[["Name", "Weighted Value", "Metacritic score"]].sort_values(by="Weighted Value", ascending=False).head(10)

,Name,Weighted Value,Metacritic score
39210,FINAL FANTASY XIV Online,114090.745373,83
271590,Grand Theft Auto V,43680.000000,96
787860,Farming Simulator 19,41240.070035,73
447020,Farming Simulator 17,40525.416945,69
440900,Conan Exiles,35084.427767,68
805550,Assetto Corsa Competizione,32131.493745,77
582010,Monster Hunter: World,31447.015672,88
72850,The Elder Scrolls V: Skyrim,30367.783892,94
107410,Arma 3,30290.896966,74
252490,Rust,28681.845461,69


In [118]:
steam_games["Release date"] = pd.to_datetime(steam_games["Release date"], errors='coerce')
steam_games.dropna(subset=["Release date"], inplace=True)

In [119]:
steam_games["Release Year"] = steam_games["Release date"].dt.year

In [120]:
price_conditions = [
    (steam_games['Price'] == 0),
    (steam_games['Price'] > 0) & (steam_games['Price'] <= 4.99),
    (steam_games['Price'] > 4.99) & (steam_games['Price'] <= 9.99),
    (steam_games['Price'] > 9.99) & (steam_games['Price'] <= 19.99),
    (steam_games['Price'] > 19.99) & (steam_games['Price'] <= 39.99),
    (steam_games['Price'] > 39.99) & (steam_games['Price'] <= 59.99),
    (steam_games['Price'] > 59.99)
]

choices = [
    'Free',
    '$0.01 - $4.99',
    '$5.00 - $9.99',
    '$10.00 - $19.99',
    '$20.00 - $39.99',
    '$40.00 - $59.99',
    '$60.00+'
]

steam_games['Price Category'] = np.select(price_conditions, choices, default="Unknown")

steam_games['Price Category'].value_counts()

Price Category
$0.01 - $4.99      43348
$5.00 - $9.99      22655
Free               16283
$10.00 - $19.99    15664
$20.00 - $39.99     4034
$40.00 - $59.99      777
$60.00+              334
Name: count, dtype: int64

In [121]:
steam_games["Achievements"].sort_values(ascending=False).head(10)

573060    9821
640310    5394
485450    5000
693890    5000
699590    5000
705090    5000
781760    5000
738000    5000
851710    5000
821520    5000
Name: Achievements, dtype: int64

In [122]:
steam_games.columns

Index(['Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age',
       'Price', 'Discount', 'DLC count', 'Supported languages',
       'Full audio languages', 'Windows', 'Mac', 'Linux', 'Metacritic score',
       'User score', 'Positive', 'Negative', 'Achievements', 'Recommendations',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Is_Game', 'Estimated owners_str',
       'Estimated owners avg', 'Value', 'Weighted Value', 'Release Year',
       'Price Category'],
      dtype='object')

In [123]:
steam_games['Steam_URL'] = 'https://store.steampowered.com/app/' + steam_games.index.astype(str) + '/'
steam_games['Steam_URL'].head(10)

20200        https://store.steampowered.com/app/20200/
655370      https://store.steampowered.com/app/655370/
1732930    https://store.steampowered.com/app/1732930/
1355720    https://store.steampowered.com/app/1355720/
1139950    https://store.steampowered.com/app/1139950/
1469160    https://store.steampowered.com/app/1469160/
1659180    https://store.steampowered.com/app/1659180/
1968760    https://store.steampowered.com/app/1968760/
1178150    https://store.steampowered.com/app/1178150/
320150      https://store.steampowered.com/app/320150/
Name: Steam_URL, dtype: object

In [124]:
steam_games["Reviews"] = steam_games["Positive"] + steam_games["Negative"]
steam_games[["Name", "Reviews", "Price", "Discount"]].sort_values(by="Reviews", ascending=False).head(10)

,Name,Reviews,Price,Discount
730,Counter-Strike: Global Offensive,6531097,0.00,0
578080,PUBG: BATTLEGROUNDS,2050633,0.00,0
570,Dota 2,1777590,0.00,0
271590,Grand Theft Auto V,1381351,30.00,0
359550,Tom Clancy's Rainbow Six® Siege,1067902,19.99,0
105600,Terraria,986027,9.99,0
440,Team Fortress 2,880376,0.00,0
4000,Garry's Mod,851330,9.99,0
252490,Rust,811910,39.99,0
2358720,Black Myth: Wukong,691809,59.99,0


In [125]:
steam_games[steam_games["Name"] == "Grand Theft Auto V"]

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,Supported languages,Full audio languages,...,Genres,Is_Game,Estimated owners_str,Estimated owners avg,Value,Weighted Value,Release Year,Price Category,Steam_URL,Reviews
271590,Grand Theft Auto V,2015-04-13,20000000 - 50000000,170527,17,30.0,0,1,"['English', 'French', 'Italian', 'German', 'Sp...","['English', 'Spanish - Latin America']",...,"Action,Adventure",True,20000000 - 50000000,35000000.0,455.0,43680.0,2015,$20.00 - $39.99,https://store.steampowered.com/app/271590/,1381351
